## overview
this note book is tells how to handle data having no cols 
1. so you need to read data in text format(it will take whole data as single col)
2. then split the data withe delimiter , ( here it will be in list format)
3. find max len of col
4. iterate cols and add new

In [0]:
#creating a csv file
dbutils.fs.put("/location/dynamic_filee.csv",
""" id,name,location,emailid
1,indra
2,sanjay,banglore
3,prasad,chennai,prasadindra17@gmail.com
4,samya,pune
""")

Wrote 106 bytes.
Out[2]: True

In [0]:
df = spark.read.csv("/location/dynamic_filee.csv",header=True)
display(df)

id,name,location,emailid
1,indra,null,null
2,sanjay,banglore,null
3,prasad,chennai,prasadindra17@gmail.com
4,samya,pune,null


In [0]:
dbutils.fs.put("/location/file_withoutcolumnss.csv","""1,indra
2,sanjay,banglore
3,prasad,chennai,prasadindra17@gmail.com
4,samya,pune""")

Wrote 79 bytes.
Out[6]: True

In [0]:
df1 = spark.read.csv("/location/file_withoutcolumnss.csv")
display(df1)

_c0,_c1
1,indra
2,sanjay
3,prasad
4,samya


In [0]:
# read file as text file you will get data and it treats all data as 1 column
df1 = spark.read.text("/location/file_withoutcolumnss.csv")
display(df1)

value
"1,indra"
"2,sanjay,banglore"
"3,prasad,chennai,prasadindra17@gmail.com"
"4,samya,pune"


In [0]:
# seperate data using split
from pyspark.sql.functions import split
df1 = df1.withColumn("splittable_col",split("value",","))
display(df1)

value,splittable_col
"1,indra","List(1, indra)"
"2,sanjay,banglore","List(2, sanjay, banglore)"
"3,prasad,chennai,prasadindra17@gmail.com","List(3, prasad, chennai, prasadindra17@gmail.com)"
"4,samya,pune","List(4, samya, pune)"


In [0]:
df1.drop('value')
display(df1)

value,splittable_col
"1,indra","List(1, indra)"
"2,sanjay,banglore","List(2, sanjay, banglore)"
"3,prasad,chennai,prasadindra17@gmail.com","List(3, prasad, chennai, prasadindra17@gmail.com)"
"4,samya,pune","List(4, samya, pune)"


In [0]:
# now creating heads/col names
#1.find max len of col
from pyspark.sql.functions import size,max
df1.select('splittable_col',size("splittable_col")).show(truncate=False)

+---------------------------------------------+--------------------+
|splittable_col                               |size(splittable_col)|
+---------------------------------------------+--------------------+
|[1, indra]                                   |2                   |
|[2, sanjay, banglore]                        |3                   |
|[3, prasad, chennai, prasadindra17@gmail.com]|4                   |
|[4, samya, pune]                             |3                   |
+---------------------------------------------+--------------------+



In [0]:
df1.select(max(size("splittable_col"))).collect()

Out[47]: [Row(max(size(splittable_col))=4)]

In [0]:
df1.select(max(size("splittable_col"))).collect()[0][0]

Out[50]: 4

In [0]:
for i in range(df1.select(max(size("splittable_col"))).collect()[0][0]):
    df1 = df1.withColumn('col'+str(i),df1['splittable_col'][i])

In [0]:
display(df1)

value,splittable_col,col0,col1,col2,col3
"1,indra","List(1, indra)",1,indra,null,null
"2,sanjay,banglore","List(2, sanjay, banglore)",2,sanjay,banglore,null
"3,prasad,chennai,prasadindra17@gmail.com","List(3, prasad, chennai, prasadindra17@gmail.com)",3,prasad,chennai,prasadindra17@gmail.com
"4,samya,pune","List(4, samya, pune)",4,samya,pune,null


In [0]:
df1 = df1.drop('value','splittable_col')

In [0]:
display(df1)

col0,col1,col2,col3
1,indra,null,null
2,sanjay,banglore,null
3,prasad,chennai,prasadindra17@gmail.com
4,samya,pune,null
